# Code to Scrape 'Movement to/from injured/inactive list (IL)'

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import os

#-----------------------------------------Defind Function ----------------------------------------
# Function to scrape NBA injury data
def scrape_nba_movement_data(start_date, end_date):
    all_data = []  # List to store data from all pages

    # Convert start_date and end_date strings to datetime objects
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.strptime(end_date, '%Y-%m-%d')

    current_date = start_date
    while current_date <= end_date:
        url = f"https://prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate={current_date.strftime('%Y-%m-%d')}&EndDate={current_date.strftime('%Y-%m-%d')}&ILChkBx=yes&Submit=Search"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extracting table data
        table = soup.find('table', {'class': 'datatable'})
        if table is not None:
            rows = table.find_all('tr')
            for row in rows[1:]:  # Skip the header row
                cols = row.find_all('td')
                # Remove bullet points from Acquired and Relinquished columns
                cols = [col.text.strip().replace('•', '') for col in cols]
                all_data.append(cols)
        
        current_date += timedelta(days=1)
#------------------------Scrape Data--------------------------------------        
    # Converting to DataFrame
    df = pd.DataFrame(all_data, columns=['Date', 'Team', 'Acquired', 'Relinquished', 'Notes'])
    return df

movement_data = scrape_nba_movement_data('2013-10-29', '2023-06-12')
if not movement_data.empty:
    print("NBA Movement to/from Injured/Inactive  Data:")
    print(movement_data)
    
    #------------------------Save Data--------------------------------------
    # define folder path
    savepath_csv = '/Users/nya/NBA project/NBA Project/00/scraped datasets'
    # Save DataFrame to CSV file
    file_path = os.path.join(savepath_csv, 'nba_movement_data.csv')
    movement_data.to_csv(file_path, index=False)
    print(f"DataFrame saved as CSV file in {file_path}.")

NBA Movement to/from Injured/Inactive  Data:
             Date      Team          Acquired       Relinquished  \
0      2013-10-29     76ers                     Jason Richardson   
1      2013-10-29     76ers                         Nerlens Noel   
2      2013-10-29  Clippers                         Maalik Wayns   
3      2013-10-29      Heat                            Greg Oden   
4      2013-10-29      Heat                      Roger Mason Jr.   
...           ...       ...               ...                ...   
17750  2023-05-25      Heat                         Gabe Vincent   
17751  2023-05-27   Celtics                      Malcolm Brogdon   
17752  2023-05-27      Heat      Gabe Vincent                      
17753  2023-05-29   Celtics   Malcolm Brogdon                      
17754  2023-06-12      Heat       Tyler Herro                      

                                                   Notes  
0      placed on IL recovering from surgery on left k...  
1      placed on IL 